In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dataset  = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
X = dataset.iloc[:, 3: -1]
y = dataset.iloc[:, -1]

In [4]:
#One_Hot_Encoding for Geography & Gender
geography = pd.get_dummies(X['Geography'], drop_first=True)
gender = pd.get_dummies(X["Gender"], drop_first=True)

In [5]:
#Concatenating The Geography & Gender to X
X = pd.concat([X,geography, gender], axis=1)
X = X.drop(["Gender", "Geography"], axis=1)
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [6]:
#Spliting The Dataset into Training & Testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
import tensorflow as tf
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, LeakyReLU, BatchNormalization
from tensorflow.keras.activations import relu, sigmoid 

In [15]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes, input_dim=X_train.shape[1]))
            model.add(activation(activation))
            model.add(Dropout(0.3))
            
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
            
    
    model.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))
    #model.add(tf.keras.layers.Dense(units=1, kernel_initializer='glorot_uniform'))
    #model.add(layers.activation('sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [18]:
model = KerasClassifier(build_fn=create_model, verbose=0)

In [16]:
from sklearn.model_selection import cross_val_score
classifier = KerasClassifier(build_fn = create_model, batch_size=100, epochs=15)

accuiracy = cross_val_score(estimator=classifier, X=X_train, y = y_train, cv=5, n_jobs=-1)

In [17]:
accuiracy

array([nan, nan, nan, nan, nan])

In [11]:
layers = [[20], [40, 20], [45,30,15]]
activations = ['sigmoid', 'relu']
param_grid  = dict(layers=layers, activation= activations, batch_size=[128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)

In [12]:
grid_result = grid.fit(X_train, y_train)
#print(grid_result.best_score_, grid_result.best_params_)

C:\Users\Sdasz\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Sdasz\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Sdasz\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\tensorflow\python\keras\wrappers\scikit_learn.py", line 223, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "C:\Users\Sdasz\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\tensorflow\python\keras\wrappers\scikit_learn.py", line 157, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
  File "<ipython-input-9-8c018b3107a1>", line 6, in create_model
    model.add(ac

RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000001A1E419C320>, as the constructor either does not set or modifies parameter layers

In [ ]:
pred_y = grid.predict(X_test)
y_pred = pred_y>0.5

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_pred, y_test)
score = accuracy_score(y_pred, y_test)